In [1]:
! pip install -r requirements.txt --quiet

# Semantic Kernel Sequential Orchestration

In sequential orchestration, agents are organized in a pipeline. Each agent processes the task in turn, passing its output to the next agent in the sequence. 

🔗 [Sequential Orchestration](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-orchestration/sequential?pivots=programming-language-python)


<img src="../../media/multi-agent-sequential.png" alt="Diagram" style="height:450px; width:auto;">


In [11]:
from semantic_kernel.agents import SequentialOrchestration
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from dotenv import load_dotenv
from os import environ
import asyncio
from user_plugins import SalesPlugin
from tracing import set_up_all

load_dotenv(override=True)

kernel = Kernel()


In [12]:
set_up_all(connection_string=environ.get("AZURE_INSIGHT_CONNECTION_STRING"))

In [13]:
chatCompletion = AzureChatCompletion(
    service_id="chat",
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    endpoint=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"],
     api_version=environ["AZURE_OPENAI_API_VERSION"] )


reasoningCompletion = AzureChatCompletion(
    service_id="reasoning",
    deployment_name='o3-mini',
    endpoint=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"],
     api_version=environ["AZURE_OPENAI_API_VERSION"] )




In [14]:

# Sales Data Agent
sales_data_agent = ChatCompletionAgent(
    name="SalesDataAgent",
    instructions=(
        """You are a Sales Data Agent. Your job is to:
        - Dynamically choose and call the appropriate tool(s) to answer user questions about sales, customers, or products.
        - Always include customer names alongside IDs in any output.
        - Always display monetary amounts in $USD (e.g., $123.45).
        - Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
        - Compute totals, revenue, discounts, and other metrics from nested order data.
        - Return **complete, detailed outputs** without summarization, so that all data is available for downstream analysis.
        - Preserve all intermediate results and raw values in your response.
        - Avoid hardcoding analytics; rely on the MCP tools and their parameters.
        - Clarify ambiguous queries before performing analysis.
        - Do not ask the user follow-up questions; only return the requested records.
        - Treat the tools as the source of truth; do not expose raw database internals.
        - return output in table format when possible for consumption by the next agent.
        """
    ),
    plugins=[SalesPlugin],
    service=chatCompletion,
)

# Data Analysis Agent
data_analysis_agent = ChatCompletionAgent(
    name="DataAnalysisAgent",
    instructions=(
        """You are a Data Analysis Agent. Given the sales data, your job is to:
        - Analyze and interpret the sales data provided by the SalesDataAgent.
        - Identify trends, patterns, and insights from the sales data.
        - Provide actionable recommendations based on the analysis.
        - Summarize key findings in a clear and concise manner.
        - Ensure that all analyses are relevant to the user's query and objectives.
        - Provide Key Findings, Insights, Trends, Recommendations and a Summary in your final output.
        """
    ),
      service=reasoningCompletion,
)

### Optional: Observe Agent Responses

In [15]:
from semantic_kernel.contents import ChatMessageContent

def agent_response_callback(message: ChatMessageContent) -> None:
    print(f"# {message.name}\n{message.content}")

### Set Up the Sequential Orchestration

In [16]:
agents = [sales_data_agent, data_analysis_agent]
sequential_orchestration = SequentialOrchestration(
    members=agents,
    agent_response_callback=agent_response_callback,
)

### Start the Runtime

In [17]:
from semantic_kernel.agents.runtime import InProcessRuntime

runtime = InProcessRuntime()
runtime.start()

# SalesDataAgent

# SalesDataAgent

# SalesDataAgent
To find the top 5 customers based on the number of orders, the sales order data is analyzed to count the orders for each customer.

### Step 1: Count Orders Per Customer
From the sales data, the following order counts per customer are determined:

| Customer ID | Customer Name | Order Count |
|-------------|---------------|-------------|
| 4           | NAPA          | 17          |
| 2           | AutoZone      | 15          |
| 1           | GM            | 13          |
| 3           | Bosch         | 7           |
| 0           | Ford          | 6           |

### Top 5 Customers
The top 5 customers by order count are:
1. **NAPA** with 17 orders
2. **AutoZone** with 15 orders
3. **GM** with 13 orders
4. **Bosch** with 7 orders
5. **Ford** with 6 orders

Let's now dig deeper into the purchasing behavior of these customers to provide actionable insights.

---

### Step 2: Analyze Purchasing Behavior
For each of the top 5 customers,

In [18]:
orchestration_result = await sequential_orchestration.invoke(
    task="Find the top 5 customers by number of orders, and bring back order and analyze their purchasing behavior to provide actionable insights for increasing sales.",
    runtime=runtime,
)

In [19]:
value = await orchestration_result.get(timeout=30)
print(f"***** Final Result *****\n{value}")

***** Final Result *****
Below is a comprehensive analysis of the top 5 customers along with key findings, insights, trends, actionable recommendations, and a summary based on the provided sales data.

─────────────────────────────  
Key Findings:
─────────────────────────────
• The top 5 customers by order count are:
  – NAPA: 17 orders  
  – AutoZone: 15 orders  
  – GM: 13 orders  
  – Bosch: 7 orders  
  – Ford: 6 orders  
• Total revenue, total quantity purchased, and preferred product categories vary among these customers.  
• Customers from regions NA and EU are present, indicating regional opportunities for targeted promotions.

─────────────────────────────  
Insights:
─────────────────────────────
• NAPA, with the highest order count, generated a revenue of $51,467.23 on 1,665 units—displaying a strong affinity for Brake Pads (804 units purchased).  
• AutoZone, earning $38,814.72 over 1,103 units, shows clear preference for Oil Filters (411 units purchased), suggesting poten

In [20]:
await runtime.stop_when_idle()